## Keras Layered Model & Explorations - JoanieW

Based on: https://elix-tech.github.io/ja/2016/06/02/kaggle-facial-keypoints-ja.html#conv

And modeled after Jackie's code structure for Lenet5

In [ ]:
import os
import imp
import pickle
import numpy as np
import pandas as pd
from utils import predict_models, load_models, transform_data
from keras.models import Sequential, Model, model_from_json

In [ ]:
cd ..

In [ ]:
### this is gold right here.
imp.reload(load_models)
file_path = "cleantrain/"
trainer = load_models.LoadTrainModels("data/models/", file_path)

trainer.print_paths()

### For every version of a clean Train file in a given path, create and save a model

In [ ]:

imp.reload(load_models)

file_path = "cleantrain/"
#files = os.listdir("../CleanTrain")
files = os.listdir(file_path)

#For every version of a cleaned Train file in CleanTrain directory, create and save a model
for filename in files: 
    print("Opening file: ", filename)
    clean_file = "".join((file_path,filename))
    print(clean_file)
    train_data = pickle.load( open( clean_file, "rb" ) )
    train_data = train_data.drop(['level_0', 'check_sum', 'index'], axis=1,errors='ignore')
    print("Train Shape:", train_data.shape)

    filename = str(filename).replace('.p', '').strip()
    print("Begin model and train:")
    model_name = "".join((filename,"_jcw"))
    print("Model name:", model_name)
    #Run with separate = True to train on 8 columns of data
    model, history = trainer.train_jcw(model_name, train_data,verbose = True, separate = True)
    print("End model and train")    
    print()

### For every model file in a given path, predict using the model and save the predictions in CSV file

In [ ]:
imp.reload(predict_models)
imp.reload(transform_data)

id_lookup = pickle.load( open( "data/id_lookup.p", "rb" ) )
test = pickle.load( open( "data/test.p", "rb" ) )

#Using local paths as this is way faster...
file_path = "data/models/"
pred_path = "data/predictions/"

predictor = predict_models.PredictModels(file_path,pred_path , id_lookup)

predictor.print_paths()

files = os.listdir(file_path)
#For every model in file_path, predict using the model and save the predictions in CSV file
for filename in files:
    if ".h5" in filename:
        base_name = filename[:-3]
        model_json = ''.join((base_name,".json"))
        print("Working with: ", base_name)
        print("Begin Predict")
        #Set columns = False if only predicting 8 keypoints
        Y= predictor.predict_jcw(base_name, filename, model_json, test, columns=False)
        print("End model and train")    
        print()


## Explorations on using different models for 8 vs. 30 predictions

In [ ]:
imp.reload(predict_models)
imp.reload(transform_data)

id_lookup = pickle.load( open( "data/id_lookup.p", "rb" ) )
test = pickle.load( open( "data/test.p", "rb" ) )

In [ ]:
#Filtering to ids where image_id count == 8
new_id = id_lookup.groupby("image_id").agg('count')
new_id_2 = new_id[new_id[ "row_id" ] <= 8]
new_id_1 = new_id[new_id[ "row_id" ] > 8]
print(new_id_2.index)
print(new_id_1.index)


In [ ]:
id_lookup_1 = id_lookup[id_lookup['image_id']<=591]
id_lookup_2 = id_lookup[id_lookup['image_id'] > 591]

In [ ]:
test_1 = test[test['image_id']<=591]
test_1

In [ ]:
test_2 = test[test['image_id']>591]
test_2

In [ ]:
pickle.dump(test_2,open("data/test_8keypoints.p", "wb"))

In [ ]:
#Identify where in the id_lookup is the last output row
print(np.max(id_lookup_1['row_id']))
print(np.min(id_lookup_1['row_id']))

max_row_full = np.max(id_lookup_1['row_id'])

The first 591 rows of the id lookups require more than 8 keypoints. Afterwards the next 1192 rows only require 6 or 8 keypoints.

In [ ]:
id_lookup = id_lookup_2
test = test_2
pred_path = "data/predictions/"

predictor = predict_models.PredictModels(file_path,pred_path , id_lookup)

model_json = "data/models/clean_w_outliers_jcw.json"
model_file = "data/models/clean_w_outliers_jcw.h5"
json_file = open(model_json, "r")
model_json_data = json_file.read()
json_file.close()
model = model_from_json(model_json_data)
model.load_weights(model_file)
base_name = "clean_w_outliers_jcw"
filename = model_file
Y= predictor.predict_jcw(base_name, filename, model_json, test)

In [ ]:
cd intro_ml/blackboxes/

## Reading in prediction csvs to combine to make final prediction file

In [ ]:
file_path = "data/predictions/"
files = os.listdir(file_path)

for file in files:
    #separate is from the most recent run of predictions based solely on train data that had 8 keypoints
    predictions_separate= pd.read_csv("data/predictions/"+file)
    #full is the older predictions data that was built on all the train
    predictions_full = pd.read_csv("data/predictions_full/"+file)
    new_pred = predictions_separate[predictions_separate['RowId']>max_row_full].copy()
    new_pred = new_pred.append(predictions_full[predictions_full['RowId']<=max_row_full].copy())
    new_pred = new_pred.sort_values(by=['RowId'])
    new_pred.to_csv("combined_"+file,index=False)
    